## **Anime recommender System**


<img src="https://cdna.artstation.com/p/assets/images/images/002/181/886/large/pixel-motron-lastsupper-animecrossover-by-pixelmotron.jpg?1458327976" width ="800">


In this notebook, we will explore creating simple recommender systems using the following techniques:

* **[Popularity Model](https://hackernoon.com/popularity-based-song-recommendation-system-without-any-library-in-python-12a4fbfd825e):** As the name suggests Popularity based recommendation system works with the trend. It basically uses the items which are in trend right now. For example, if any product which is usually bought by every new user then there are chances that it may suggest that item to the user who just signed up. The problem is that, it might not give personalized recommendations.This model works well most of the times because people like things other people might like. For e.g: Many people have watched Dragon Ball Z. There are so many pop culture references about DBZ that people who generally dont like anime might have watched/ would prefer watching the show.



* **[Content Based Filtering Algorithm:](http://http://recommender-systems.org/content-based-filtering/)** This method will try to find similarities between the content ( Anime series in this case) using attributes and metadata about the content and try to recommend items with highest similarity to what our active user liked in the past.



* **[Collaborative filtering algorithm](https://en.wikipedia.org/wiki/Collaborative_filtering):** This method will try to find similar users like the active user (the user for whom we are going to make predictions). Based on certain similarity metrics, we then suggest items ( anime titles in our case) that the similar users liked(gave higher ratings to) which were not rated by our active users.


(Inspired by Gabriel Moreira's Kaggle Notebook on Recommender Systems)


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.display.max_rows = 4000
from sklearn.metrics.pairwise import cosine_similarity
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# Read data
anime_data= pd.read_csv('/kaggle/input/anime-recommendations-database/anime.csv')
rating_data=pd.read_csv('/kaggle/input/anime-recommendations-database/rating.csv')

## Exploring our Data

In [ ]:
# Checking basic information about our dataset
anime_data.head()

In [ ]:
anime_data.describe()
# The data consists of 12294 different Anime titles, it has 12064 ratings (which means some ratings are Null)

In [ ]:
rating_data.head()
#This data contains Iser ID, Anime Id and User rating for that anime from 0 to 10.
#Items which have a rating of -1 have not been rated yet.

In [ ]:
rating_data.describe()
# Rating data has 7.8 Million Ratings

In [ ]:
# replacing -1 to np.Nan in rating data
rating_data.rating.replace(-1, np.NaN,inplace=True)

In [ ]:
# Getting count of Nulls
print('Anime Data')
for i in anime_data.columns:
    print('Null counts in the column',i,':',sum(anime_data[i].isna()))

print('\n Rating Data')
for i in rating_data.columns:
    print('Null counts in the column',i,':',sum(rating_data[i].isna()))

Getting so many NAs in ratings is understandable and a very common issue in User-Rating data. 
This is because, not every user will rate every anime in the list of anime shows out there!

In [ ]:
# replacing -1 to np.Nan in rating data
anime_data.genre.replace(np.NaN,'None_Genre',inplace=True)
anime_data.type.replace(np.NaN,'None_type',inplace=True)
anime_data.episodes.replace('Unknown',np.NaN,inplace=True)

## EDA

**EDA on anime_data **

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import itertools
import collections
import operator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale

In [ ]:
# On average, they have a rating of 6
sns.distplot(anime_data['rating'], hist=True, kde=True, 
             bins=10, color = 'green', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
print('Average Rating:',anime_data['rating'].mean())

In [ ]:
# On average, they have a rating of 6
sns.distplot(anime_data['members'], hist=True, kde=True, 
             bins=10, color = 'green', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
print('Average members:',anime_data['members'].mean())
print('Max members:',anime_data['members'].max())
print('Min members:',anime_data['members'].min())
print('Median members:',anime_data['members'].median())

In [ ]:
## Check the top 10 anime with max members
anime_data.sort_values('members',ascending=False).head(10)

In [ ]:
## Check the bottom 10 anime titles
anime_data.sort_values('members',ascending=True).head(10)

Many of these anime titles dont even have a Rating / Genre (I am not surprised! )

In [ ]:
anime_data.episodes=pd.to_numeric(anime_data.episodes, errors='coerce')

In [ ]:
# On average, they have aroud 12 episodes per series
sns.distplot(anime_data['episodes'], hist=True, kde=True, 
             bins=10, color = 'green', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
print('Average Episode Count:',anime_data['episodes'].mean())
print('Max Episode Count:',anime_data['episodes'].max())
print('Min Episode Count:',anime_data['episodes'].min())
print('Median Episode Count:',anime_data['episodes'].median())

In [ ]:
# Anime series with highest number of episodes
anime_data.sort_values('episodes', ascending = False).head(10)

In [ ]:
anime_data['type'].value_counts()

In [ ]:
# Checking rating across different type of anime
sns.catplot(x="type", y="rating", kind="box", data=anime_data)

for i in anime_data['type'].unique():
    print('Average rating for',i,'anime:',anime_data[anime_data.type==i]['rating'].mean())

 We notice that, on an average rating for TV anime is higher ~ 7
 
 Average rating for Musical Anime is lower ~ 5.5


In [ ]:
anime_data['genre']=anime_data['genre'].apply(lambda x : x.split(', '))

In [ ]:
genre_data = itertools.chain(*anime_data['genre'].values.tolist())
genre_counter = collections.Counter(genre_data)
genres = pd.DataFrame.from_dict(genre_counter,orient='index').reset_index()
genres.columns=['Genre','Counts']
genres.sort_values('Counts', ascending=False, inplace=True)

# Plot genre
f, ax = plt.subplots(figsize=(10,12))
sns.barplot(x="Counts", y="Genre", data=genres, color='#719967')
ax.set(ylabel='Genre',xlabel="Count")

Our list mostly comprises of comedy anime titles, followed by Action, adventure and fantasy titles

In [ ]:
# Getting rankings by genre
genre_rating = []
for i in list(genres['Genre']):
    genre_rating.append(anime_data[anime_data['genre'].str.contains(i, regex=False)]['rating'].mean())

genre_rating_dict=pd.DataFrame({'Genre': list(genres['Genre']),
  'rating': genre_rating })
genre_rating_dict.sort_values('rating', ascending=False, inplace=True)

In [ ]:
# Plot Genre - Ratings 
f, ax = plt.subplots(figsize=(10,12))
sns.barplot(x="rating", y="Genre", data=genre_rating_dict, color='#719967')
ax.set(ylabel='Genre',xlabel="Rating")


Not much variation in average ratings for different genres. 

* Josei anime tends to have a higher rating averaging at 7 points. This can also because these anime titles have lower number of titles.

* Dementia anime series are at the tail end of ratings and average on around 5 points.

* Most of the remaining anime titles are averaging between 6-7 points 

**EDA on Rating dataset**

In [ ]:
rating_anime=rating_data.merge(anime_data[['name','genre','anime_id','type','episodes','members']],left_on='anime_id',right_on='anime_id')

In [ ]:
## Lets look at top 10 animes which have been rated the most in the dataset
top_rated= rating_anime.groupby(['anime_id','name']).count()['user_id'].reset_index().sort_values('user_id', ascending=False)
top_rated.head(10)

We see that the top 10 numbers are some of the higher and more mainstream anime titles like Naruto, Code Geass and Attack on Titan (Shingeki no Kyojin). This list does look similar to the list in the "anime_data" table

In [ ]:
sns.distplot(top_rated['user_id'], hist=True, kde=True, 
             bins=10, color = 'green', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

**Long Tail Plot:** The above plot shows what is called the Long tail phenomenon. This plot is used to explore popularity patterns in user-item interaction data such as clicks, ratings, or purchases. Typically, only a small percentage of items have a high volume of interactions, and this is referred to as the “head”. Most items are in the “long tail”, but they only make up a small percentage of interactions. 


[Reference](http://https://towardsdatascience.com/evaluation-metrics-for-recommender-systems-df56c6611093)

In [ ]:
#http://sdsawtelle.github.io/blog/output/mean-average-precision-MAP-for-recommender-systems.html

### Now that we have seen how our data looks like, let's Build our Recommender Systems!!!

## **Popularity Based Recommender System** 
* For our popularity based recommender system, we will be writing a function which takes the following parameters:
    * **N**= Number of Shows to recommend
    
    * **Genre** = A list of genres that the recommender system should recommend out of. By default, it will take all genre into consideration
    
    * **Type** = A list of the type of show that the user wants (e.g TV, Movie, OVA etc). By default, it will take all types into consideration
    
    * **episodes_more_than** an integer value which indicates how many integers that particular anime show should be more than. e.g for a value of 50, the system should only show the shows which have more than 50 episodes. by default, it is set to 0.
    * **Popularity Weight** = A value between 0 to 1.  A value of 1 means all the recommendations will be based on the popularity of the show. A value of 0 means all the recommendations will be based on the rating for the show. A value of 0.6 means, the popularity will be given a 60% weightage and 40% weightage will be given to the rating of a show

In [ ]:
def popularity_recommender(dataset,N = 0, 
                           Genre =[],
                           Type= [],
                           episodes_more_than=0,
                           popularity_weight= 0.5
                          ):    
    if len(Genre)==0:
        Genre = ['Josei', 'Thriller', 'Mystery', 'Police', 'Shounen', 'Psychological', 'Military', 'Supernatural', 'Romance', 'Shoujo Ai', 'Drama', 'School', 'Seinen', 'Harem', 'Shounen Ai', 'Super Power', 'Vampire', 'Shoujo', 'Samurai', 'Martial Arts', 'Magic', 'Action', 'Game', 'Sports', 'Historical', 'Adventure', 'Slice of Life', 'Sci-Fi', 'Space', 'Demons', 'Fantasy', 'Ecchi', 'Mecha', 'Comedy', 'Parody', 'Cars', 'Yaoi', 'Horror', 'Hentai', 'Kids', 'Yuri', 'Music', 'None_Genre', 'Dementia']
    if len(Type) ==0:
        Type = ['ONA', 'None_type', 'OVA', 'Special', 'Music', 'Movie', 'TV']

    pop_recommender_df = dataset[
    (dataset.episodes>=episodes_more_than) &
    (dataset.genre.apply(len) !=(dataset.genre.apply(set)- set(Genre)).apply(len)) &
    (dataset.type.isin(Type)) ].copy()
       
    if len(pop_recommender_df)==0:
        print('No anime found with such conditions')

    else:
        # anime rating is in a range of 1 to 10 while popularity/members is in a larger range. 
        # because we want to show good shows with higher popularity, we will scale popularity on a range of 1 to 10 
        # we shall be giving 70% weight to the popularity metric and 30% weight to show rating and calculate a new score called Popularity_quality_index
        pop_recommender_df['scaled_members']=(minmax_scale(pop_recommender_df.members))*10
        pop_recommender_df['Popularity_quality_index']=(popularity_weight* pop_recommender_df.scaled_members)+(1-popularity_weight)*pop_recommender_df.rating
        df=pop_recommender_df.sort_values('Popularity_quality_index', ascending=False).iloc[0:N]
        cols =['name','genre','type','episodes','rating','members']
        return(df[cols])


In [ ]:
genre_list=['Action']
Type_list = ['TV']
popularity_recommender(anime_data,
                       Genre=genre_list,
                       episodes_more_than=0,
                       Type=Type_list,
                       N=20,
                       popularity_weight=0.6)

Looks really good! I can see some of the popular Anime titles in this list. Because we have the ability to assign weights and tune our list of recommendations, and subset our results, its a very easy to use and intuitive solution.

However, this method wont give you personalized recommendations.

## Data Cleaning and creating training dataset

In [ ]:
# Getting rid of NA ratings
print('Original rating data size:',rating_data.shape[0])
rating_data_clean = rating_data[rating_data.rating.notna()].reset_index(drop = True).copy()
# Getting rid of Users with less than 250 ratings
print('Rating data after removing NA Ratings:',rating_data_clean.shape[0])
# Getting rid of Anime titles with less than 250 ratings 
anime_rating_counts = pd.DataFrame(rating_data_clean.groupby('anime_id')['user_id'].nunique()).reset_index()
req_anime_ids= anime_rating_counts[anime_rating_counts['user_id']>250].anime_id
user_rating_counts=pd.DataFrame(rating_data_clean.groupby('user_id')['anime_id'].nunique()).reset_index()
req_user_ids= user_rating_counts[user_rating_counts['anime_id']>250].user_id
# Splitting data into training and test sets
rating_clean=rating_data_clean[rating_data_clean.anime_id.isin(req_anime_ids) &
                              rating_data_clean.user_id.isin(req_user_ids)].copy().reset_index(drop = True)
print('Rating_data after filtering Anime Shows and Users:',rating_clean.shape[0])

In [ ]:
# Split into training and test
train_df, test_df = train_test_split(rating_clean,
                                   test_size=0.20,
                                   random_state=27)

# Content Based Recommender System

In [ ]:
anime_genre_dummies= pd.get_dummies(anime_data.genre.apply(pd.Series).stack()).sum(level=0)
anime_genre_dummies=pd.concat([anime_data, anime_genre_dummies], axis=1)


In [ ]:
def get_episode_encoding(num_episodes):
    if(num_episodes<=13):
        return('Xsmall')
    elif (num_episodes<=50):
        return('Small')
    elif (num_episodes<=250):
        return('Medium')
    elif (num_episodes<=500):
        return('Long')
    else:
        return('Xlong')

In [ ]:
# Engineering a few features

#Making one hot encodings for type, Number of Episodes, and if the show is a very popular show, has medium popularity or lesser known popularity
# Getting Show Type dummies
anime_genre_dummies = pd.concat([anime_genre_dummies,pd.get_dummies(anime_data.type)],axis = 1)

# Getting Episode size categories
anime_genre_dummies = pd.concat([anime_genre_dummies,pd.get_dummies(anime_genre_dummies.episodes.apply(get_episode_encoding))],axis=1)

req_cols = list(set(anime_genre_dummies.columns) -set(['anime_id','name','genre','type','episodes','rating','members']))
req_cols.sort()

# Creating Encodings
anime_genre_dummies['Encoding']=anime_genre_dummies[req_cols].values.tolist()
anime_genre_dummies

This system will take the list of anime that the active user has watched. Based on the list of anime, we will try to find similar items in our list of anime. We are using Cosine similarity to find similarities between the active user's profile and the titles listed in our database

The function takes the following parameters:
* **user_profile** = This will be the list of anime shows rated by our user
* **N** = Number of recommendations to suggest to the Active User
* **movie_profile ** = Content database

In [ ]:
def content_based_recommender(user_id,N,movie_profile):
    # get user profile
    user_profile=rating_clean[rating_clean['user_id']==user_id].copy().reset_index(drop = True)
    user_profile = user_profile[user_profile.rating.notna()]
    cols_to_merge= list(set(movie_profile.columns)-set(['rating']))
    
    user_profile =pd.merge(user_profile[['anime_id','rating']],movie_profile[cols_to_merge]
             ,how= 'left', left_on='anime_id', right_on='anime_id')
    
    req_cols = list(set(movie_profile.columns) -set(['anime_id','name','genre','type','episodes','rating','members','Encoding']))
    req_cols.sort()
    # to generate the user profile, we are summing up the genre dummy variables that the user
    #interacted with and weighing it with the rating that the user has given to the interacted items 
    user_profile['rating_scaled'] =  minmax_scale(user_profile['rating'])
    
    # Multiplying Ratings with the genre encodings
    genre_weights=pd.DataFrame(user_profile[req_cols].multiply(user_profile['rating_scaled'],axis="index").sum()).reset_index()
    genre_weights.columns=['Genre','Weights']
    
    #Scaling the encodings so that we have encodings in 0-1 range to compare with movie encodings
    genre_weights['weights_scaled']=minmax_scale(genre_weights['Weights'])
    user_profile_weights= [list(genre_weights['weights_scaled'])]
    
    # Finding cosine similarity between user profile and movies 
    movie_profile['user_affinity']= cosine_similarity(user_profile_weights,list(movie_profile['Encoding']))[0]
    
    return(movie_profile.sort_values('user_affinity', ascending = False).reset_index().iloc[1:N][['name','genre','type','episodes','rating','members','user_affinity']])


In [ ]:
rating_clean.user_id.unique()[1:15]

In [ ]:
content_based_recommender(user_id =17, N= 10,movie_profile = anime_genre_dummies.copy())

Based on the recommendations for user 17, we can see that the user really likes Action,Comedy and High School based Anime Shows (Mostly TV).

Note that the Content based Recommender System is not giving highly rated or popular anime series. This recommender system will provide items similar to the content that the user has seen and rated currently.
Factors like: Genre, type of Anime and Number of episodes are the factors that the recommender system uses to find similar other content!

# Collaborative Filtering using the 'Surprise' Package

In [ ]:
import surprise
from surprise.model_selection import cross_validate
from surprise import SVD,SVDpp,NMF,NormalPredictor,KNNBaseline,KNNBasic,KNNWithMeans,KNNWithZScore,BaselineOnly,CoClustering,SlopeOne, Reader, Dataset
# we will be using the Surprise Python Package to get our recommendations.
# We will try a few different Models for implementing Collaborative filtering. 

ratings_dict = {'itemID': rating_clean.anime_id,
                'userID': rating_clean.user_id,
                'rating': rating_clean.rating}
df = pd.DataFrame(ratings_dict)
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 10))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)
import time 
start = time.time()


benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), NMF()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=True, n_jobs = 1)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')  
end = time.time()
print(end - start)


** Looks like the SVD model has the least RMSE Error. We will be using this model and fine tune it further over the whole dataset to see if we can improve the model performance **

In [ ]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# Train an SVD algorithm on the Anime Rating cleaned dataset.

ratings_dict = {'itemID': rating_clean.anime_id,
                'userID': rating_clean.user_id,
                'rating': rating_clean.rating}
df = pd.DataFrame(ratings_dict)
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 10))
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

# Precision and recall @ k

In [ ]:
from collections import defaultdict

from surprise import Dataset
from surprise import SVD
from surprise.model_selection import KFold


def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=7)
# Precision and recall can then be averaged over all users
print(sum(prec for prec in precisions.values()) / len(precisions))
print(sum(rec for rec in recalls.values()) / len(recalls))

In [ ]:
# Lets use an example user

In [ ]:
uid_test=list(top_n.keys())

In [ ]:
def get_recommendations(uid_profile):
    user_top_10 = rating_anime[rating_anime.user_id == uid_profile].sort_values('rating',ascending = False ).iloc[1:10]
    rated_top_10= user_top_10[['name','genre','type','episodes','rating']]
    anime_id_user= [iid for (iid, _) in top_n[uid_profile]]
    top_10_recommendations = anime_data[anime_data.anime_id.isin(anime_id_user)][['name','genre','type','episodes','rating']]
    return rated_top_10,top_10_recommendations

In [ ]:
uid_test[3]

In [ ]:
# Getting user ratings and recommendations for user number 38

user_rated,top_10_recommendations =  get_recommendations(uid_profile=38)

In [ ]:
print('these are the top 10 anime titles rated by user 38')
user_rated

In [ ]:
print('these are our recommendations for the user 38')
top_10_recommendations